# Week 3: Task 5-8
#### Name: Kai Ferragallo-Hawkins
#### Date: 30.1.2023

## #Setup

While seeding with the random package is an option for random sampling (to initialize the random number generator), I decided to instead use the built-in Pandas sampling code.

In [90]:
from matplotlib import pyplot as plt
import pandas as pd
import math
import numpy as np
import samplics as smp
import seaborn as sns
import random as rdm
import SurveySamplingFunctions as ssf

### Province 91
## Importing Province91 Data
province91 = pd.read_csv("files/assignment1/province91.txt", delim_whitespace=True)

## Random Sampling with Pandas: random_state for consistent randomness, no replacement
prov91_sample = province91.sample(n=8, replace=False, random_state=123456)

# Displaying the newly created sample
display(prov91_sample)

### Province 17
## Importing Province17 Data
province17 = pd.read_csv("files/assignment1/province17.txt", delimiter = '\t', encoding='latin-1')

## Random Sampling with Pandas: random_state for consistent randomness, no replacement.
## Fraction kept the same for random sampling.
prov17_sample = province17.sample(n=8, replace=False, random_state=123456)

# Displaying the newly created sample
display(prov17_sample)

,Stratum,Cluster,Id,Municipality,POP91,LAB91,UE91,HOU85,URB85
31,2,8,32,Viitasaari,8641,4011,568,3119,0
21,2,6,22,Luhanka,1153,522,54,435,0
23,2,1,24,Muurame,6830,3024,296,1853,0
3,1,2,4,Keuruu,12707,5919,760,4896,1
9,2,7,10,Jyväskmlk,29349,13727,1623,9230,0
5,1,5,6,Suolahti,6159,3022,457,2389,1
22,2,7,23,Multia,2375,1059,119,925,0
16,2,1,17,Korpilahti,5181,2144,239,1793,0


,Municipality,id,POP17,LAB17,UE17,HOU17,URB17_new
22,Äänekoski,23,19144,7239,1128,9352,1
3,Jämsä,4,20877,7989,1175,10647,1
14,Multia,15,1636,570,59,776,0
9,Konnevesi,10,2748,1022,134,1339,0
5,Karstula,6,4146,1468,227,1974,1
19,Toivakka,20,2384,956,106,1070,0
13,Luhanka,14,734,244,26,372,0
6,Keuruu,7,9919,3517,493,4970,1


## #Task 5: Calculate total, variance, and confidence interval from SRSWOR Sample

In [103]:
### Province 91
## Calculating the Total of UE91
prov91_weight = len(province91)/len(prov91_sample)
#print(prov91_weight)
#print(prov91_sample['UE91'].sum())
#total_ue91 = province91['UE91'].sum()
total_ue91 = prov91_weight*prov91_sample['UE91'].sum()

## Calculating the System Variance of UE91 (sqrt)
# Done by manually calculating the previous information for design variance, multiplied by the population variance as given by Pandas. ddof=0 makes equation give population variance.

def system_var (main, sample, sample_key, Is_Sqrt = False):
    """"Calculates the system variance by manually calculating the information for design variance based on population sizes, given two pandas dataframes of the original information and the sample. Population variance calculated through Pandas var."""
    sqr = 0.5 if Is_Sqrt else 1
    s_var = ((len(main)**2)*(1-len(sample)/len(main))*(1/len(sample))*sample[sample_key].var(ddof=0))**sqr
    return s_var

s_var91 = system_var(province91, prov91_sample, "UE91", Is_Sqrt = True)

## Calculating the Confidence Interval of UE91
# Done with 95%, using np array to do +- in the same line.
high_int_91, low_int_91 = prov91_sample["UE91"].mean() + np.array([+1, -1])*1.96*prov91_sample["UE91"].std()/math.sqrt(prov91_sample["UE91"].count())

### Province 17
## Calculating the Total of UE17
prov17_weight = len(province17)/len(prov17_sample)
total_ue17 = prov17_weight*prov17_sample['UE17'].sum()

## Calculating the System Variance of UE17 (sqrt)
s_var17 = system_var(province17, prov17_sample, "UE17", Is_Sqrt = True)

## Calculating the Confidence Interval of UE17
high_int_17, low_int_17 = prov17_sample["UE17"].mean() + np.array([+1, -1])*1.96*prov17_sample["UE17"].std()/math.sqrt(prov17_sample["UE17"].count())

### Both
display(pd.DataFrame({'Year': ["1991", "2017"],'Total Unemployed, Absolute': [total_ue91, total_ue17], 'System Variance (sqrt)': [round(s_var91, 2), round(s_var17, 2)], 'Mean Unemployed': [prov91_sample["UE91"].mean(), prov17_sample["UE17"].mean()], 'Lower 95% Interval': [round(low_int_91), round(low_int_17)], 'Upper 95% Interval': [round(high_int_91), round(high_int_17)]}))

,Year,"Total Unemployed, Absolute",System Variance (sqrt),Mean Unemployed,Lower 95% Interval,Upper 95% Interval
0,1991,16464.0,4631.20,514.5,164,865
1,2017,9625.5,2918.24,418.5,89,748


## #Task 6: Ratio (Unemployment Rate), S.E. for Ratio, and Median for UE91 and Lab 91

In [122]:
### Province 91
## Calculating Ratio Unemployment Rate
# Individual unemployment rates for each municipality
prov91_sample["Unemployed % in 1991"] = round(prov91_sample["UE91"]/prov91_sample["LAB91"]*100, 2)
display(prov91_sample[["Municipality", "Unemployed % in 1991"]])

## Calculating Standard Error of Unemployment Rate
uer_std_err91 = prov91_sample["Unemployed % in 1991"].sem()

## Median of UE91 and LAB91
UE91_med = prov91_sample["UE91"].median()
LAB91_med = prov91_sample["LAB91"].median()

### Province 17
## Calculating Ratio Unemployment Rate
prov17_sample["Unemployed % in 2017"] = round(prov17_sample["UE17"]/prov17_sample["LAB17"]*100, 2)
display(prov17_sample[["Municipality", "Unemployed % in 2017"]])

## Calculating Standard Error of Unemployment Rate
uer_std_err17 = prov17_sample["Unemployed % in 2017"].sem()

## Median of UE17 and LAB17
UE17_med = prov17_sample["UE17"].median()
LAB17_med = prov17_sample["LAB17"].median()

### Both
display(pd.DataFrame({'Year': ["1991", "2017"],'Unemployment Rate, Standard Error': [uer_std_err91, uer_std_err17], 'Unemployment Median': [UE91_med, UE17_med], 'Labor Median': [LAB91_med, LAB17_med]}))


,Municipality,Unemployed % in 1991
31,Viitasaari,14.16
21,Luhanka,10.34
23,Muurame,9.79
3,Keuruu,12.84
9,Jyväskmlk,11.82
5,Suolahti,15.12
22,Multia,11.24
16,Korpilahti,11.15


,Municipality,Unemployed % in 2017
22,Äänekoski,15.58
3,Jämsä,14.71
14,Multia,10.35
9,Konnevesi,13.11
5,Karstula,15.46
19,Toivakka,11.09
13,Luhanka,10.66
6,Keuruu,14.02


,Year,"Unemployment Rate, Standard Error",Unemployment Median,Labor Median
0,1991,0.655354,376.5,3023.0
1,2017,0.764278,180.5,1245.0


## #Task 7: Design Effect (Deff) for total, ratio (unemployment rate), and median

In [123]:
### Province 91
## Calculating UE91 Total Deff
deff_total_ue91 = province91["UE91"].sum()/total_ue91

## Calculating UE Ratio Total Deff
province91["Unemployed % in 1991"] = round(province91["UE91"]/province91["LAB91"]*100, 2)
deff_ratio_ue91 = province91["Unemployed % in 1991"].sem()/uer_std_err17

## Median UE and LAB Deff
deff_median_ue91 = province91["UE91"].median()/UE91_med
deff_median_lab91 = province91["LAB91"].median()/LAB91_med

### Province 17
## Calculating UE17 Total Deff
deff_total_ue17 = province17["UE17"].sum()/total_ue17

## Calculating UE Ratio Total Deff
province17["Unemployed % in 2017"] = round(province17["UE17"]/province17["LAB17"]*100, 2)
deff_ratio_ue17 = province17["Unemployed % in 2017"].sem()/uer_std_err17

## Median UE and LAB Deff
deff_median_ue17 = province17["UE17"].median()/UE17_med
deff_median_lab17 = province17["LAB17"].median()/LAB17_med

### Both
display(pd.DataFrame({'Year': ["1991", "2017"],'Deff, UE Total': [deff_total_ue91, deff_total_ue17],'Deff, UE Rate %': [deff_ratio_ue91, deff_ratio_ue17],'Deff, UE Median': [deff_median_ue91, deff_median_ue17],'Deff, LAB Median': [deff_median_lab91, deff_median_lab17]}))

,Year,"Deff, UE Total","Deff, UE Rate %","Deff, UE Median","Deff, LAB Median"
0,1991,0.917031,0.612214,0.608234,0.629507
1,2017,1.585788,0.683319,1.113573,1.243373


## #Task 8: Commenting on the Simulation Examples

When looking at the VLISS Training Key 28 B data, the most notable trend is how the range of the UE to LAB ratios (0.159 n=4 -> 0.126 n=32) and confidence intervals (0.129 - 0.1897 n=4 -> 0.126 - 0.126 n=32) decreased over all counts of n. This comes from the increasing accuracy that a larger sample size provides compared to the base population. However, different samples, especially low samples, can have quirks that break a trend, like how n=4's standard errors for it's UE and LAB totals was the second lowest, behind only n=32's at 0.

VLISS Training Key 28 C 

## # Reflection - Comparing Province91 with Province17

When looking at the above table that compares all 1991 and 2017 values, we can see that the Unemployment rate did increase, but just barely. It is small enough (0.8% absolute change, or a 6.4% increase in 1991 vs 2017) that there is not much interesting information that it can provide alone. However, the variance/standard deviation - with a change from 743 to 1705, or a 129% increase - shows that the distribution from 1991 to 2017 did change significantly, meaning that on a region by region difference there are much clearer cases of higher and lower unemployment numbers. This may be due more to population than %, however, as the box plot shows that there are strong outliers by region.